# Time Series Forecasting Model for Tesla Stock Prices
This notebook focuses on building a time series forecasting model to predict Tesla's future stock prices.
The steps include:
1. Splitting the data into training and testing sets.
2. Building and training different time series forecasting models (ARIMA, SARIMA, LSTM).
3. Optimizing the model parameters.
4. Evaluating model performance with metrics like MAE, RMSE, and MAPE.
---

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
import matplotlib.pyplot as pltgit
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# LSTM requirements
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator


In [ ]:
# Load the Tesla stock data
tesla_data = pd.read_csv('../data/tesla_data.csv', index_col='Date', parse_dates=True)
tesla_data = tesla_data[['Close']]  # Use only 'Close' prices for forecasting

# Display the first few rows of the data
tesla_data.head()

In [ ]:
# Split the data into training and testing sets
train_size = int(len(tesla_data) * 0.8)
train, test = tesla_data[:train_size], tesla_data[train_size:]

# Plot the training and test data
plt.figure(figsize=(12, 6))
plt.plot(train, label='Training Data')
plt.plot(test, label='Testing Data')
plt.title('Tesla Stock Price')
plt.xlabel('Date')
plt.ylabel('Normalized Close Price')
plt.legend()
plt.show()

In [ ]:
# Define a function to evaluate model performance
def evaluate_forecast(true, predicted):
    mae = mean_absolute_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    mape = mean_absolute_percentage_error(true, predicted)
    return {'MAE': mae, 'RMSE': rmse, 'MAPE': mape}

In [ ]:
# ARIMA Model
from pmdarima import auto_arima

# Finding the best (p, d, q) parameters
arima_model = auto_arima(train, seasonal=False, trace=True, error_action='ignore', suppress_warnings=True, stepwise=True)

# Train ARIMA model
model = ARIMA(train, order=arima_model.order)
arima_result = model.fit()

# Forecast
arima_forecast = arima_result.predict(start=len(train), end=len(tesla_data)-1, dynamic=False)

# Evaluate ARIMA
arima_eval = evaluate_forecast(test, arima_forecast)
print("ARIMA Evaluation:", arima_eval)

# Plot ARIMA results
plt.figure(figsize=(12, 6))
plt.plot(train, label='Training Data')
plt.plot(test, label='Testing Data')
plt.plot(test.index, arima_forecast, label='ARIMA Forecast')
plt.title('ARIMA Model - Tesla Stock Price Forecast')
plt.xlabel('Date')
plt.ylabel('Normalized Close Price')
plt.legend()
plt.show()

In [ ]:
# SARIMA Model
sarima_model = auto_arima(train, seasonal=True, m=12, trace=True, error_action='ignore', suppress_warnings=True, stepwise=True)

# Train SARIMA model
model = SARIMAX(train, order=sarima_model.order, seasonal_order=sarima_model.seasonal_order)
sarima_result = model.fit(disp=False)

# Forecast
sarima_forecast = sarima_result.predict(start=len(train), end=len(tesla_data)-1, dynamic=False)

# Evaluate SARIMA
sarima_eval = evaluate_forecast(test, sarima_forecast)
print("SARIMA Evaluation:", sarima_eval)

# Plot SARIMA results
plt.figure(figsize=(12, 6))
plt.plot(train, label='Training Data')
plt.plot(test, label='Testing Data')
plt.plot(test.index, sarima_forecast, label='SARIMA Forecast')
plt.title('SARIMA Model - Tesla Stock Price Forecast')
plt.xlabel('Date')
plt.ylabel('Normalized Close Price')
plt.legend()
plt.show()

In [ ]:
# LSTM Model
from sklearn.preprocessing import MinMaxScaler 
# Prepare the data for LSTM model
scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train)
test_scaled = scaler.transform(test)

# Define generator
n_input = 30
n_features = 1
generator = TimeseriesGenerator(train_scaled, train_scaled, length=n_input, batch_size=32)

# Build LSTM model
lstm_model = Sequential([
    LSTM(50, activation='relu', input_shape=(n_input, n_features)),
    Dense(1)
])
lstm_model.compile(optimizer='adam', loss='mse')

# Train LSTM model
lstm_model.fit(generator, epochs=20)

# Forecast
lstm_forecast = []
first_eval_batch = train_scaled[-n_input:]
current_batch = first_eval_batch.reshape((1, n_input, n_features))

for i in range(len(test)):
    lstm_pred = lstm_model.predict(current_batch)[0]
    lstm_forecast.append(lstm_pred)
    current_batch = np.append(current_batch[:, 1:, :], [[lstm_pred]], axis=1)

# Inverse transform to get actual prices
lstm_forecast = scaler.inverse_transform(lstm_forecast)

# Evaluate LSTM
lstm_eval = evaluate_forecast(test.values, lstm_forecast)
print("LSTM Evaluation:", lstm_eval)

# Plot LSTM results
plt.figure(figsize=(12, 6))
plt.plot(train, label='Training Data')
plt.plot(test, label='Testing Data')
plt.plot(test.index, lstm_forecast, label='LSTM Forecast')
plt.title('LSTM Model - Tesla Stock Price Forecast')
plt.xlabel('Date')
plt.ylabel('Normalized Close Price')
plt.legend()
plt.show()